In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import MistralConfig, MistralForCausalLM, AutoTokenizer

[2023-11-02 06:05:42,310] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
!ls fpf-7b-32k

checkpoint-7400  checkpoint-7500


In [4]:
half_config = MistralConfig.from_pretrained('fpf-7b-32k/checkpoint-7500', 
                                             num_hidden_layers = 4)
half_config

MistralConfig {
  "_name_or_path": "mesolitica/mistral-7b-4096-fpf",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 4,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.34.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [5]:
small = MistralForCausalLM(half_config)

In [6]:
sum(p.numel() for p in small.parameters())

1134596096

In [7]:
model = MistralForCausalLM.from_pretrained('fpf-7b-32k/checkpoint-7500')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
import torch.nn as nn

def copy_layers(src_layers, dest_layers, layers_to_copy):
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

layers_to_copy = [0,1,2,3]

copy_layers(model.model.layers, small.model.layers, layers_to_copy)

In [11]:
small.model.embed_tokens.load_state_dict(model.model.embed_tokens.state_dict())

<All keys matched successfully>

In [12]:
small.model.norm.load_state_dict(model.model.norm.state_dict())

<All keys matched successfully>

In [13]:
small.lm_head.load_state_dict(model.lm_head.state_dict())

<All keys matched successfully>

In [14]:
small.save_pretrained("1b")

In [15]:
tokenizer = AutoTokenizer.from_pretrained('fpf-7b-32k/checkpoint-7500')
tokenizer.save_pretrained("1b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


('1b/tokenizer_config.json',
 '1b/special_tokens_map.json',
 '1b/tokenizer.model',
 '1b/added_tokens.json',
 '1b/tokenizer.json')

In [16]:
!ls -lh 1b

total 4.3G
-rw-r--r-- 1 ubuntu ubuntu   42 Nov  2 06:11 added_tokens.json
-rw-r--r-- 1 ubuntu ubuntu  617 Nov  2 06:11 config.json
-rw-r--r-- 1 ubuntu ubuntu  111 Nov  2 06:11 generation_config.json
-rw-r--r-- 1 ubuntu ubuntu 4.3G Nov  2 06:11 pytorch_model.bin
-rw-r--r-- 1 ubuntu ubuntu  145 Nov  2 06:11 special_tokens_map.json
-rw-r--r-- 1 ubuntu ubuntu 1.8M Nov  2 06:11 tokenizer.json
-rw-r--r-- 1 ubuntu ubuntu 482K Nov  2 06:11 tokenizer.model
-rw-r--r-- 1 ubuntu ubuntu  953 Nov  2 06:11 tokenizer_config.json
